# ICANN 2025 Hackathon

Now that you have followed the presentation (see [its notebook here](./01_Introduction.ipynb)), it's time for you to play with the library!

In [ ]:
%pip install --quiet reservoirpy[hyper]==0.4.* dash

In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np

from reservoirpy import ESN
from reservoirpy.nodes import Reservoir, Ridge, ES2N, IPReservoir, NVAR
from reservoirpy.observables import nrmse, rsquare

In [ ]:
# Dataset definition
from reservoirpy.datasets import lorenz
timeseries = lorenz(1020)[20:]

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(timeseries)
plt.title("Lorenz timeseries")
plt.subplot(1, 2, 2, projection="3d")
plt.plot(timeseries[:,0], timeseries[:,1], timeseries[:,2])
plt.show()

In [ ]:
# Rescale
mins = timeseries.min(axis=0)
maxs = timeseries.max(axis=0)
timeseries = (timeseries - mins) / (maxs - mins)

In [ ]:
# Create a dataset from it
from reservoirpy.datasets import to_forecasting
dataset = to_forecasting(timeseries, forecast=10, test_size=0.2)
x_train, x_test, y_train, y_test = dataset

# Play around with *ReservoirPy*

In [ ]:
# Create your model here

# You can play with the different nodes: 
# IPReservoir (intrinsic plasticity), NVAR (next-generation RC), LocalPlasticityReservoir (synaptic plasticity), ...
# You can also compose nodes together to create a more complex model (DeepESN, hierarchical ESN, ...)
model = ...

In [ ]:
# Train and run
model.fit(x_train, y_train)
y_pred = model.run(x_test)

In [ ]:
# Evaluate predictions
print(f"NRMSE = {nrmse(y_test, y_pred, dimensionwise=True)}")

# Optimize hyperparameters

## **Step 1**: Define the objective

In [ ]:
# Define the dataset to use:
dataset = x_train, x_test, y_train, y_test

In [ ]:
# Objective functions accepted by ReservoirPy must respect some conventions:
#  - dataset and config arguments are mandatory, like the empty '*' expression.
#  - all parameters that will be used during the search must be placed after the *.
#  - the function must return a dict with at least a 'loss' key containing the result of the loss function.
# You can add any additional metrics or information with other keys in the dict. See hyperopt documentation for more informations.
def objective(dataset, config, *, input_scaling, N, sr, lr, ridge, seed):
    # This step may vary depending on what you put inside 'dataset'
    x_train, x_test, y_train, y_test = dataset

    # You can access anything you put in the config file from the 'config' parameter.
    instances = config["instances_per_trial"]

    # The seed should be changed across the instances to be sure there is no bias in the results due to initialization.
    variable_seed = seed

    losses = []; r2s = [];
    for n in range(instances):
        # Build your model given the input parameters
        model = ESN(units=200)
        # raise NotImplementedError("Remove this error and put your model here!")

        # Train your model and test your model.
        predictions = model.fit(x_train, y_train) \
                           .run(x_test)

        loss = nrmse(y_test, predictions, norm_value=np.ptp(x_train))
        r2 = rsquare(y_test, predictions)

        # Change the seed between instances
        variable_seed += 1

        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}

## **Step 2**: Define the research space

In [ ]:
import json

hyperopt_config = {
    "exp": "hyperopt-1",    # the experimentation name
    "hp_max_evals": 200,              # the number of differents sets of parameters hyperopt has to try
    "hp_method": "random",            # the method used by hyperopt to chose those sets (see below)
    "seed": 42,                       # the random state seed, to ensure reproducibility
    "instances_per_trial": 1,         # how many random ESN will be tried with each sets of parameters
    "hp_space": {                     # what are the ranges of parameters explored
        "N": ["choice", 100],             # the number of neurons is fixed to 100
        "sr": ["loguniform", 1e-5, 1e2],   # the spectral radius is log-uniformly distributed from 1e-5 and 100
        "lr": ["loguniform", 1e-5, 1.0],    # idem with the leaking rate, from 1e-5 to 1
        "input_scaling": ["choice", 1.0],   # the input scaling is fixed
        "ridge": ["loguniform", 1e-6, 1e6],        # regularization parameter is explored widely
        "seed": ["randint", 0, 2**32]          # seeds used for random initialisation of weights matrices
    }
}

# we precautionously save the configuration in a JSON file
# each file will begin with a number corresponding to the current experimentation run number.
with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
    json.dump(hyperopt_config, f)

## **Step 3**: Launch the hyperparameter search

In [ ]:
from reservoirpy.hyper import parallel_research
best = parallel_research(objective, dataset, f"{hyperopt_config['exp']}.config.json", ".")

## **Step 4**: Plot report

#### Static method using *ReservoirPy*

In [ ]:
from reservoirpy.hyper import plot_hyperopt_report
fig = plot_hyperopt_report(hyperopt_config["exp"], ("lr", "sr", "ridge"), metric="r2")

#### Interactive report

In [ ]:
from interactive_report import interactive_report
interactive_report(hyperopt_config['exp'], ['lr', 'sr', 'ridge'], log_loss=True, out_path='limit.json')

## **Step 5**: Refine the search space

Let's refine the search space based on the new ranges we have found for the hyperparameters.
And let's now allow the input scaling to vary to potentially find better optimum.

In [ ]:
import json

hyperopt_config = {
    "exp": "hyperopt-doublescroll-refined-01",    # the experimentation name changed (so we can save the results in another folder)
    "hp_max_evals": 200,              
    "hp_method": "random",           
    "seed": 42,                    
    "instances_per_trial": 1,       
    "hp_space": {                   
        "N": ["choice", 100],            
        "sr": ["loguniform", 1e-5, 1e2],    # we keep the SR ranges unchanged because the ranges seem already good and don't change much the loss
        "lr": ["loguniform", 10**(-2.34), 1.0],    # change to new find range values 
        "input_scaling": ["loguniform", 1e-5, 1e2],    # making the input scaling variable: we take the same values as SR
        "ridge": ["loguniform", 1e-10, 10**(-0.32)],       # we reduced the range on the right-hand part and augment it on the left-hand part
        "seed": ["randint", 0, 2**32]          # an other random seed for the ESN initialization
    }
}

# we precautionously save the configuration in a JSON file
# each file will begin with a number corresponding to the current experimentation run number.
with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
    json.dump(hyperopt_config, f)

In [ ]:
best = parallel_research(objective, dataset, f"{hyperopt_config['exp']}.config.json", ".")

In [ ]:
fig = plot_hyperopt_report(hyperopt_config["exp"], ("lr", "sr", "ridge", "input_scaling"), metric="r2")